In [ ]:
import threading
import time

class BakeryAlgorithm:
    def __init__(self, num_processes):
        self.num_processes = num_processes
        self.choosing = [False] * num_processes
        self.tickets = [0] * num_processes

    def lock(self, process_id):
        self.choosing[process_id] = True
        self.tickets[process_id] = max(self.tickets) + 1
        self.choosing[process_id] = False

        for i in range(self.num_processes):
            if i == process_id:
                continue
            while self.choosing[i]:
                pass
            while self.tickets[i] != 0 and (
                (self.tickets[i] < self.tickets[process_id]) or
                (self.tickets[i] == self.tickets[process_id] and i < process_id)
            ):
                pass

    def unlock(self, process_id):
        self.tickets[process_id] = 0


def compute_partial_factorial(start, end, process_id, bakery, shared_resource):
    """Compute the product from `start` to `end`."""
    bakery.lock(process_id)

    print(f"Thread {process_id} computing product from {start} to {end}.")
    partial_result = 1
    for i in range(start, end + 1):
        partial_result *= i
    shared_resource[process_id] = partial_result
    print(f"Thread {process_id} completed: partial result = {partial_result}.")

    bakery.unlock(process_id)


number = 50  # The number to compute the factorial of
num_threads = 5
step = number // num_threads  # Split the range equally among threads

shared_resource = [1] * num_threads  # Store partial results
bakery = BakeryAlgorithm(num_threads)

threads = []
for i in range(num_threads):
    start = i * step + 1
    end = (i + 1) * step if i < num_threads - 1 else number
    t = threading.Thread(target=compute_partial_factorial, args=(start, end, i, bakery, shared_resource))
    threads.append(t)
    t.start()

# Wait for all threads to finish
for t in threads:
    t.join()

# Compute the final factorial by multiplying all partial results
final_result = 1
for partial_result in shared_resource:
    final_result *= partial_result

print(f"\nThe factorial of {number} is {final_result}.")
